In [ ]:
# --- train_model.ipynb ---
import pandas as pd, numpy as np, matplotlib.pyplot as plt, os
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


In [ ]:
# Cell 1: EDA
df = pd.read_csv('raw_data.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)
df.plot(subplots=True, figsize=(15, 12))
plt.show()

In [ ]:
# Cell 2: Preprocessing
df.fillna(method='ffill', inplace=True)
df_smooth = df.rolling(window=5).mean().dropna()

In [ ]:
# Cell 3: Model Training
def create_windows(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:(i + window_size)]); y.append(data[i + window_size])
    return np.array(X), np.array(y)

In [ ]:
healthy_cutoff = int(len(df_smooth) * 0.6)
df_healthy = df_smooth.iloc[:healthy_cutoff]
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_healthy[['az']])
WINDOW_SIZE = 50
X_train, y_train = create_windows(scaled_data, WINDOW_SIZE)
model = Sequential([ LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True), Dropout(0.2), LSTM(32), Dropout(0.2), Dense(1) ])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)
os.makedirs('models', exist_ok=True)
model.save('models/time_series_model.h5')
train_pred = model.predict(X_train)
train_mae_loss = np.mean(np.abs(train_pred - y_train), axis=1)
anomaly_threshold = np.max(train_mae_loss) * 1.5
np.save('models/anomaly_threshold.npy', anomaly_threshold)
print(f"Model and threshold saved. Anomaly Threshold: {anomaly_threshold}")
